# Браки и разводы.
Проект является экспериментальной лабораторной работой.

## Цели.
1. Чтение и анализ xml файлов ЕМИСС.
2. Визуализация результаов анализа с помощью Altair.
3. Сторителлинг на основе визуального анализа.
4. Генерация html контента на основе сторителлинга.

## Чтение и анализ xml файлов ЕМИСС.

In [97]:
# Загружаем библиотеки
import pandas as pd

Для исследования источника я выбрал файл в котором есть десятичные значения. Буду проверять не будет ли проблем при их чтении. Спойлер — будут!

In [98]:
# Создаём ссылку на xml файл
#data_raw='Число зарегистрированных браков (оперативные данные).xml'
data_raw='Число зарегистрированных разводов в расчете на 1000 населения (оперативные данные).xml'

### Что внутри xml.

Для чтения используется функция read_xml. Посмотрев xml файл можно увидеть, что есть несколько элементов, на которые можно сослаться:
1. Коды регионов.
2. Информация о переменной.
3. Год.
4. Значения переменной.

#### Коды регионов.

Для чтения я использую функцию [read_xml](https://pandas.pydata.org/docs/reference/api/pandas.read_xml.html)

Пример использования, который мне помог:

Если файл выглядит так

```xml
xml = '''<?xml version='1.0' encoding='utf-8'?>
     <doc:data xmlns:doc="https://example.com">
       <doc:row>
         <doc:shape>square</doc:shape>
         <doc:degrees>360</doc:degrees>
         <doc:sides>4.0</doc:sides>
       </doc:row>
       <doc:row>
         <doc:shape>circle</doc:shape>
         <doc:degrees>360</doc:degrees>
         <doc:sides/>
       </doc:row>
       <doc:row>
         <doc:shape>triangle</doc:shape>
         <doc:degrees>180</doc:degrees>
         <doc:sides>3.0</doc:sides>
       </doc:row>
     </doc:data>'''
```

Тогда читать его нужно вот так

```python
df = pd.read_xml(xml,
                xpath="//doc:row",
                namespaces={"doc": "https://example.com"})
df
```

Результат

```
      shape  degrees  sides
0    square      360    4.0
1    circle      360    NaN
2  triangle      180    3.0
```
[Почитать по теме ещё](https://medium.com/@robertopreste/from-xml-to-pandas-dataframes-9292980b1c1c)

In [99]:
# Получение данных кодов регионов

namespaces_codes = 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure' # Имя объекта, где хранятся коды регионов

regiones_codes = pd.read_xml(data_raw, # Ссылка на файл
                            xpath="//structure:Code", # Путь внутри файла до данных
                            namespaces = {"structure": namespaces_codes}) # Имя объекта
regiones_codes['value'] = regiones_codes['value'].astype(str) # Преобразую колонку с кодами в текстовой формат. Это нужно будет для консолидации в будущем
regiones_codes # Результат

,value,Description
0,643,Российская Федерация
1,30,Центральный федеральный округ
2,14000000000,Белгородская область
3,15000000000,Брянская область
4,17000000000,Владимирская область
...,...,...
104,99000000000,Еврейская автономная область
105,77000000000,Чукотский автономный округ
106,39,Крымский федеральный округ
107,46001000000,Московская обл. в старых границах


#### Информация о переменной.
Действую по аналогии с предыдущей таблицей. Но уже новый источник внутри xml файла и результат хоть и похожий, но мы не можем его в таком виде использовать.

In [100]:
namespaces_values = 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic'

regiones_values = pd.read_xml(
    data_raw, xpath="//generic:Value", 
    namespaces = {"generic": namespaces_values}
    )
regiones_values

,concept,value
0,s_OKATO,643
1,EI,"промилле (0,1 процента)"
2,PERIOD,январь
3,s_OKATO,643
4,EI,"промилле (0,1 процента)"
...,...,...
71152,EI,"промилле (0,1 процента)"
71153,PERIOD,декабрь
71154,s_OKATO,45001000000
71155,EI,"промилле (0,1 процента)"


Полученные данные идут подряд в определённой последовательности. Нужно первый столбец сделать названием колонок. Для начала нам нужен список колонок.

In [101]:
concept_list = regiones_values['concept'].drop_duplicates() # Забираю список концептов и удалаю дубликаты
concept_list

0    s_OKATO
1         EI
2     PERIOD
Name: concept, dtype: object

Нужен цикл, который действует следующим образом:
1. Берёт имя переменной из списка
2. Назначает это столбцу новой таблицы имя выбранной переменной
3. Заполняет столбец данными, которые соответствуют этому имени

> Важно помнить, что количество строк в получившейся таблице должно совпадать с другими таблицами, которые мы будем создавать из новых источников позже.

In [102]:
regiones_values_pivot = pd.DataFrame() # Новая пустая таблица

# Цикл преобразования плоской таблицы в сводную
for i in concept_list:
    regiones_values_pivot[i] = regiones_values.loc[regiones_values['concept'] == i]['value'].values

regiones_values_pivot

,s_OKATO,EI,PERIOD
0,643,"промилле (0,1 процента)",январь
1,643,"промилле (0,1 процента)",январь-февраль
2,643,"промилле (0,1 процента)",январь-март
3,643,"промилле (0,1 процента)",январь-апрель
4,643,"промилле (0,1 процента)",январь-май
...,...,...,...
23714,45001000000,"промилле (0,1 процента)",январь-октябрь
23715,45001000000,"промилле (0,1 процента)",ноябрь
23716,45001000000,"промилле (0,1 процента)",январь-ноябрь
23717,45001000000,"промилле (0,1 процента)",декабрь


#### Год
Число года записано проще прочих данных.

In [103]:
regiones_Obs = pd.read_xml(
    data_raw, 
    xpath="//generic:Obs", 
    namespaces = {"generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic"}
    )
regiones_Obs

,Time,ObsValue
0,2006,NaN
1,2006,NaN
2,2006,NaN
3,2006,NaN
4,2006,NaN
...,...,...
23714,2013,NaN
23715,2013,NaN
23716,2013,NaN
23717,2013,NaN


#### Значения.
Значения выглядят, как длинный список цифр десятичного формата

In [104]:
regiones_ObsValue = pd.read_xml(data_raw, xpath="//generic:ObsValue", namespaces = {"generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic"})
regiones_ObsValue

,value
0,"3,41"
1,"3,77"
2,"4,17"
3,"4,24"
4,"4,34"
...,...
23714,0
23715,0
23716,0
23717,0


In [105]:
# Нужно проверить, что за тип данных у нас внутри.
regiones_ObsValue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23719 entries, 0 to 23718
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   value   23719 non-null  object
dtypes: object(1)
memory usage: 185.4+ KB


Видно, что колонка value имеет cмешанный тип, хотя должен быть int или float. Почему так? Из-за запятых в качестве десятичного разделителя.

> Важно помнить, что разделитель десятичного значения должен быть точкой!

Нужно заменить запятую на точку

In [106]:
regiones_ObsValue2 = regiones_ObsValue
regiones_ObsValue2 ['value2'] = regiones_ObsValue['value'].str.replace(',','.').replace('.',None).astype(float)

В процессе преобразования я выяснил, что в данных также есть ячейки, где есть текстовые значения в виде точек. Их тоже пришлось заменить на пустое значение

In [107]:
regiones_ObsValue2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23719 entries, 0 to 23718
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   value   23719 non-null  object 
 1   value2  23719 non-null  float64
dtypes: float64(1), object(1)
memory usage: 370.7+ KB


Проверка показывает, что теперь у нас есть список чисел, а не смешанный набор данных. Ура!

## Сведение и консолидация

Теперь самое интерсное: сводим имеющиеся таблицы.

concat - сводит таблицы с одинаковым индексом добавляя колонки справа
merge - консолидирует две таблицы по указанным столбцам

In [108]:
data = pd.concat([
                regiones_values_pivot, # Первая таблица с данными значений
                regiones_Obs['Time'], # Столбец из таблицы со значениями года
                regiones_ObsValue2['value2'] # Столбец из таблицы статистических значений
                ],
                axis=1 # Указываем, что столбцы мы объединяем по горизонтали
            ).merge(regiones_codes, # Добавляем таблицу со списком регионов
                    left_on='s_OKATO', # Указываем столбец по котоорму будет вестись консолидация в левой исходной таблице
                    right_on='value', # Указываем в каком столбце лежат данные для консолидации в правой присоединяемой таблице
            ).drop(columns='value') # Удаляем столбец из правой таблицы, который дублирует столбец левой таблицы
data

,s_OKATO,EI,PERIOD,Time,value2,Description
0,643,"промилле (0,1 процента)",январь,2006,3.41,Российская Федерация
1,643,"промилле (0,1 процента)",январь-февраль,2006,3.77,Российская Федерация
2,643,"промилле (0,1 процента)",январь-март,2006,4.17,Российская Федерация
3,643,"промилле (0,1 процента)",январь-апрель,2006,4.24,Российская Федерация
4,643,"промилле (0,1 процента)",январь-май,2006,4.34,Российская Федерация
...,...,...,...,...,...,...
20135,45001000000,"промилле (0,1 процента)",январь-октябрь,2013,0.00,Москва в старых границах
20136,45001000000,"промилле (0,1 процента)",ноябрь,2013,0.00,Москва в старых границах
20137,45001000000,"промилле (0,1 процента)",январь-ноябрь,2013,0.00,Москва в старых границах
20138,45001000000,"промилле (0,1 процента)",декабрь,2013,0.00,Москва в старых границах


На этом я остановлюсь, чтобы поделиться опытом. Далее нужно из этого сделать функцию и осуществлять создание таблиц из нескольких источников, чтобы, например сравнивать браки и разводы в регионах атоматически